Instructor code: https://colab.research.google.com/drive/1oEjHwGfHVdY_Sx8w4BLEuBihORnWJ2w1?usp=sharing#scrollTo=gWr05mfxVBqH

In [224]:
import numpy as np
import string
import random

In [411]:
poems = []

file = 'robert_frost.txt'
for line in open(file,encoding='utf-8'):
    line = line.rstrip().lower()
    if line:
        line = line.translate(str.maketrans('', '', string.punctuation))
        poems.append(line)

#### Get starting word, first-order transition and second-order transition dictionaries 

In [412]:
#Dictionary for first word
pi = {}
for variable in [sentence.split()[0] for sentence in poems]:
    if variable in pi:
        pi[variable] += 1
    else:
        pi[variable] = 1
pi

{'two': 8,
 'and': 129,
 'to': 50,
 'then': 12,
 'because': 1,
 'though': 7,
 'had': 4,
 'in': 29,
 'oh': 4,
 'yet': 3,
 'i': 118,
 'somewhere': 1,
 'whose': 2,
 'his': 7,
 'he': 34,
 'my': 7,
 'between': 3,
 'the': 82,
 'of': 29,
 'but': 51,
 'some': 5,
 'from': 10,
 'is': 5,
 'natures': 1,
 'her': 2,
 'so': 13,
 'nothing': 2,
 'when': 9,
 'came': 1,
 'one': 11,
 'proclaimed': 1,
 'smoothlaid': 1,
 'half': 3,
 'up': 7,
 'a': 30,
 'disturbed': 2,
 'comes': 1,
 'by': 11,
 'if': 12,
 'were': 3,
 'ten': 1,
 'as': 14,
 'it': 23,
 'lifted': 1,
 'she': 17,
 'transfixed': 1,
 'across': 1,
 'was': 9,
 'once': 2,
 'that': 29,
 'looked': 1,
 'stood': 1,
 'you': 40,
 'me': 1,
 'sometimes': 4,
 'brown': 2,
 'cross': 1,
 'describing': 1,
 'got': 1,
 'walls': 1,
 'like': 8,
 'upon': 2,
 'with': 18,
 'faster': 1,
 'sitting': 1,
 'according': 1,
 'hes': 11,
 'or': 11,
 'incredulous': 1,
 'well—i—be—': 1,
 'on': 11,
 'should': 2,
 'yankees': 1,
 'dont': 7,
 'until': 2,
 'after': 1,
 'not': 13,
 'at': 6

In [413]:
#Dictionary for first order transition
A1 = {}
for f,s in [tuple(sentence.split()[0:2]) for sentence in poems if len(sentence.split())>1]:  
    if f in A1: 
        if s in A1[f]:
            A1[f][s] += 1
        else:
            A1[f].update({s:1})
    else:
        A1[f] = {s:1}

In [415]:
#Dictionary for second order transitions 
A2 = {}
for sentence in poems:
    words = sentence.split()
    W = len(words)
    if(W>2):
        w = 0
        while w<W-2:
            w0 = words[w]
            w1 = words[w+1]
            w2 = words[w+2]
            if w0 in A2:
                if w1 in A2[w0]:
                    A2[w0][w1][w2] = A2[w0][w1].get(w2,0) + 1 
                else:
                    A2[w0][w1] = {w2:1}
            else:
                A2[w0] = {w1:{w2:1}}
            w += 1

#### Get probabilities from the counts

factor=1.0/sum(d.itervalues())
normalised_d = {k: v*factor for k, v in d.iteritems() }

https://stackoverflow.com/questions/16417916/normalizing-dictionary-values/16418021 

In [416]:
pi_prob = {}
factor = 1/sum(pi.values())
pi_prob = {k:v*factor for k,v in pi.items()}

#Check that it worked
sum(list(pi_prob.values()))
pi_prob['that']

0.0201949860724234

In [417]:
#[item for sublist in t for item in sublist]
A1_prob = {}
#factor = 1/sum([counts for first_word in A1.values() for counts in first_word.values()])
A1_prob = {outer_k: {inner_k: inner_v*1/sum(outer_v.values()) for inner_k, inner_v in outer_v.items()} for outer_k, outer_v in A1.items()}

#Check that it worked
sum(A1_prob['and'].values())

1.000000000000001

In [418]:
A2_prob = {}
#factor =1/sum([counts for first_word in A2.values() for second_word in first_word.values() for counts in second_word.values()])

A2_prob = {o2_k: 
           {o1_k: 
            {in_k:in_v*1/sum(o1_v.values()) for in_k, in_v in o1_v.items()} 
                for o1_k, o1_v in o2_v.items()} 
                for o2_k, o2_v in A2.items()}

#Check that it worked
sum(A2_prob['and']['sorry'].values())

1.0

#### Generate poem

In [419]:
def gen_word(p, prob):
    lower = 0
    upper = 0
    for k, v in prob.items():
        upper += v
        if lower < p and p < upper: 
            return k
        lower = upper

In [420]:
def write_poetry():
    for i in range(4):
        sentence = []
        p = random.uniform(0,1)
        w0 = gen_word(p,pi_prob)
        sentence.append(w0)
        if A1_prob.get(w0,0) == 0: 
            break
        w1 = gen_word(p,A1_prob[w0])
        sentence.append(w1)
        while A2_prob.get(w0,0) != 0 and A2_prob[w0].get(w1,0) != 0 and len(sentence) <= 10:
            w2 = gen_word(p,A2_prob[w0][w1])
            sentence.append(w2)
            w0 = w1
            w1 = w2 
        print(' '.join(sentence))

In [462]:
write_poetry()

bad to get away
and miles to go before i sleep
its whirling frenzies
well then its all up i think he will do is
